In [2]:
data = [
    {"id": "1", "name": "bulbasaur", "size": {"weight": 6.9, "height": 0.7}},
    {"id": "4", "name": "charmander", "size": {"weight": 8.5, "height": 0.6}},
    {"id": "25", "name": "pikachu", "size": {"weight": 6, "height": 0.4}},
]

data

[{'id': '1', 'name': 'bulbasaur', 'size': {'weight': 6.9, 'height': 0.7}},
 {'id': '4', 'name': 'charmander', 'size': {'weight': 8.5, 'height': 0.6}},
 {'id': '25', 'name': 'pikachu', 'size': {'weight': 6, 'height': 0.4}}]

In [3]:
import dlt

# Set pipeline name, destination, and dataset name
pipeline = dlt.pipeline(
    pipeline_name="quick_start",
    destination="duckdb",
    dataset_name="mydata",
    dev_mode = True,
    progress="log",
)

You instantiate a pipeline by calling the dlt.pipeline function with the following arguments:

**pipeline_name**: This is the name you give to your pipeline. It helps you track and monitor your pipeline, and also helps to bring back its state and data structures for future runs. If you don't give a name, dlt will use the name of the Python file you're running as the pipeline name.

**destination**: a name of the destination to which dlt will load the data. It may also be provided to the run method of the pipeline.
dataset_name: This is the name of the group of tables (or dataset) where your data will be sent. You can think of a dataset like a folder that holds many files, or a schema in a relational database. You can also specify this later when you run or load the pipeline. If you don't provide a name, it will default to the name of your pipeline.

**dev_mode**: If you set this to True, dlt will add a timestamp to your dataset name every time you create a pipeline. This means a new dataset will be created each time you create a pipeline.
There are more arguments, but they are for advanced use, we skip it for now.

In [4]:
# Run the pipeline with data and table name
load_info = pipeline.run(data, write_disposition="replace", table_name="pokemon")

print(load_info)

----------------------------- Extract quick_start ------------------------------
Resources: 0/1 (0.0%) | Time: 0.00s | Rate: 0.00/s
Memory usage: 174.47 MB (65.10%) | CPU usage: 0.00%

----------------------------- Extract quick_start ------------------------------
Resources: 0/1 (0.0%) | Time: 0.00s | Rate: 0.00/s
pokemon: 1  | Time: 0.00s | Rate: 349525.33/s
Memory usage: 174.52 MB (65.10%) | CPU usage: 0.00%

----------------------------- Extract quick_start ------------------------------
Resources: 1/1 (100.0%) | Time: 0.01s | Rate: 72.41/s
pokemon: 3  | Time: 0.01s | Rate: 238.86/s
Memory usage: 174.52 MB (65.10%) | CPU usage: 0.00%

----------------------------- Extract quick_start ------------------------------
Resources: 0/1 (0.0%) | Time: 0.00s | Rate: 0.00/s
Memory usage: 174.55 MB (65.10%) | CPU usage: 0.00%

----------------------------- Extract quick_start ------------------------------
Resources: 0/1 (0.0%) | Time: 0.00s | Rate: 0.00/s
_dlt_pipeline_state: 1  | Time: 0.00

Commonly used arguments:

* **data** (the first argument) may be a dlt source, resource, generator function, or any Iterator or Iterable (i.e., a list or the result of the map function).
* **write_disposition** controls how to write data to a table. Defaults to "append".
    * `append` will always add new data at the end of the table.
    * `replace` will replace existing data with new data.
    * `skip` will prevent data from loading.
    * `merge` will deduplicate and merge data based on primary_key and merge_key hints.
* **table_name**: specified in cases when the table name cannot be inferred, i.e., from the resources or name of the generator function.

In [ ]:
# You can quickly inspect the generated tables, the data, see how many rows were loaded to which table,
# do SQL queries, etc., by executing the following command from the same folder as your script

# streamlit should be installed
%%capture
uv add streamlit


%%capture
dlt pipeline quick_start show

Use the pipeline name you defined in your Python code with the pipeline_name argument. If you are unsure, you can use the following command to list all pipelines:

In [ ]:
dlt pipeline --list

# Explore the loaded data:


---
### **(1) DuckDB Connection**

Start a connection to your database using native duckdb connection and look what tables were generated:

In [5]:
import duckdb

# Connect to the DuckDB database
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# Set search path to the dataset
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# Describe the dataset
conn.sql("DESCRIBE").df()

,database,schema,name,column_names,column_types,temporary
0,quick_start,mydata_20241227020710,_dlt_loads,"[load_id, schema_name, status, inserted_at, sc...","[VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...",False
1,quick_start,mydata_20241227020710,_dlt_pipeline_state,"[version, engine_version, pipeline_name, state...","[BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...",False
2,quick_start,mydata_20241227020710,_dlt_version,"[version, engine_version, inserted_at, schema_...","[BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...",False
3,quick_start,mydata_20241227020710,pokemon,"[id, name, size__weight, size__height, _dlt_lo...","[VARCHAR, VARCHAR, DOUBLE, DOUBLE, VARCHAR, VA...",False


You can see:
-  `pokemon` table,

and 3 special `dlt` tables (we will discuss them later):
- `_dlt_loads`,
- `_dlt_pipeline_state`,
- `_dlt_version`.

Let's execute a query to get all data from the `pokemon` table:

In [7]:
# Fetch all data from 'pokemon' as a DataFrame
table = conn.sql("SELECT * FROM pokemon").df()

# Display the DataFrame
table

,id,name,size__weight,size__height,_dlt_load_id,_dlt_id
0,1,bulbasaur,6.9,0.7,1735308434.035593,cs6TssI83RzuBw
1,4,charmander,8.5,0.6,1735308434.035593,Ab4oNZ6NilLpnw
2,25,pikachu,6.0,0.4,1735308434.035593,IKyX5JlTDzi6eQ


 ---
 ### **(2) `dlt`'s [sql_client](https://dlthub.com/docs/general-usage/dataset-access/sql-client)**

Most dlt destinations (even filesystem) use an implementation of the `SqlClientBase` class to connect to the physical destination to which your data is loaded. You can access the SQL client of your destination via the `sql_client` method on your pipeline.

Start a connection to your database with `pipeline.sql_client()` and execute a query to get all data from the `pokemon` table:

In [8]:
# Query data from 'pokemon' using the SQL client
with pipeline.sql_client() as client:
    with client.execute_query("SELECT * FROM pokemon") as cursor:
        data = cursor.df()

# Display the data
data

,id,name,size__weight,size__height,_dlt_load_id,_dlt_id
0,1,bulbasaur,6.9,0.7,1735308434.035593,cs6TssI83RzuBw
1,4,charmander,8.5,0.6,1735308434.035593,Ab4oNZ6NilLpnw
2,25,pikachu,6.0,0.4,1735308434.035593,IKyX5JlTDzi6eQ


---
### **(3) dlt [datasets](https://dlthub.com/docs/general-usage/dataset-access/dataset)**

Here's an example of how to retrieve data from a pipeline and load it into a Pandas DataFrame or a PyArrow Table.

In [19]:
dataset = pipeline.dataset(dataset_type="default")
pandas_df = dataset.pokemon.df()

pandas_df

,id,name,size__weight,size__height,_dlt_load_id,_dlt_id
0,1,bulbasaur,6.9,0.7,1735308434.035593,cs6TssI83RzuBw
1,4,charmander,8.5,0.6,1735308434.035593,Ab4oNZ6NilLpnw
2,25,pikachu,6.0,0.4,1735308434.035593,IKyX5JlTDzi6eQ


---
# **Exercise 1**

Using the code from the previous cell, fetch the data from the `pokemon` table into a dataframe and count the number of columns in the table `pokemon`.


In [20]:
len(pandas_df.columns)

6